In [3]:
import imgaug.augmenters as iaa
import imgaug as ia
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import pydensecrf.densecrf as dcrf

In [3]:
patches = np.load('patches_3.npz')
fg_patches = patches['fg_patches']
bg_patches = patches['bg_patches']
boxes = np.load('bboxes_3.npz')

In [82]:
bbox_on_images = [ia.BoundingBoxesOnImage([ia.BoundingBox(box[0],box[1],box[0] + box[2],box[1] + box[3],label = box[-1]) for _,box in enumerate(boxes)],shape = (299,299)) for i,boxes in enumerate(boxes.values())]
crop = iaa.CropToFixedSize(256,256,position = 'center')
reordered_bbs = [np.array([np.array((b.y1,b.x1,b.y2,b.x2))for b in bb]) for bb in bbs]
labels = [np.array([b.label for b in bb]) for bb in bbs]

In [94]:
np.savez_compressed('training_imgs.npz',fg_patches = fg_imgs,bg_patches = bg_imgs)

In [95]:
np.savez_compressed('box_labels.npz',*labels)

In [96]:
np.savez_compressed('box_coords.npz',*reordered_bbs)

In [77]:
patches = np.load('patches_1.npz')
boxes = np.load('bboxes_1.npz')
fg_patches = patches['fg_patches']
bg_patches = patches['bg_patches']
np.savez_compressed('val_imgs.npz',fg_patches = orig_fg,bg_patches = orig_bg)
np.savez_compressed('val_box_labels.npz',*labels)
np.savez_compressed('val_box_coords.npz',*reordered_bbs)

In [43]:
aug = iaa.Sequential([iaa.Rot90([0,1,2,3]),iaa.Sometimes(.5,iaa.OneOf([iaa.Fliplr(),iaa.Flipud()])),iaa.Sometimes(.5,iaa.Affine(translate_px={"x": (-16, 16), "y": (-4, 4)},rotate=(-10,10))),iaa.Sometimes(.75,iaa.ElasticTransformation(alpha = (.2,1),sigma = .02)),iaa.Sometimes(.5,iaa.AddToHueAndSaturation((-5,5)))])
n = 5
aug_det = aug.to_deterministic(n)
all_bbs = []
fg_imgs = []
bg_imgs = []
for i in range(n):
    this_aug = aug_det[i]
    fg_img = this_aug.augment_images(fg_patches.astype(np.uint8))
    fg_img = crop.augment_images(fg_img)
    bg_img = this_aug.augment_images(bg_patches.astype(np.uint8))
    bg_img = crop.augment_images(bg_img)    
    bbs = []
    for j,box in enumerate(bbox_on_images):
      box = this_aug.augment_bounding_boxes(box)
      aug_bb = crop.augment_bounding_boxes(box)
      if aug_bb.remove_out_of_image().empty:
        continue
      else:
        bbs.append(aug_bb.remove_out_of_image().clip_out_of_image().bounding_boxes)
    all_bbs += bbs
    fg_imgs.append(fg_img)
    bg_imgs.append(bg_img)

In [37]:
box = this_aug.augment_bounding_boxes(box)
aug_bb = crop.augment_bounding_boxes(box)
reordered_bbs = [np.array([np.array((b.y1,b.x1,b.y2,b.x2))for b in bb]) for bb in bbs]
labels = [np.array([b.label for b in bb]) for bb in bbs]

In [80]:
bbs = []
for j,box in enumerate(bbox_on_images):
  #box = this_aug.augment_bounding_boxes(box)
  aug_bb = crop.augment_bounding_boxes(box)
  if aug_bb.remove_out_of_image().empty:
    continue
  else:
    bbs.append(aug_bb.remove_out_of_image().clip_out_of_image().bounding_boxes)
reordered_bbs = [np.array([np.array((b.y1,b.x1,b.y2,b.x2))for b in bb]) for bb in bbs + all_bbs]
labels = [np.array([b.label for b in bb]) for bb in bbs + all_bbs]

In [86]:
orig_fg = crop.augment_images(fg_patches)
orig_bg = crop.augment_images(bg_patches)

In [93]:
reordered_bbs

[array([[ 71.0975,  92.4475, 106.5925, 126.29  ]]),
 array([[ 53.0825,  21.5425, 110.235 ,  72.2025],
        [145.5275,  53.6125, 235.    ,  97.5525],
        [  7.6175, 220.2825,  66.07  , 235.    ],
        [217.3975,  75.45  , 235.    , 117.015 ]]),
 array([[ 40.0275,  16.97  , 111.6675,  51.3575],
        [  0.    , 229.025 ,  65.5   , 235.    ]]),
 array([[ 63.09  ,  91.2225,  92.7625, 107.2   ],
        [  1.0575, 230.45  ,  21.0325, 235.    ],
        [ 95.9825,  72.99  , 112.8025,  88.225 ]]),
 array([[ 96.5925,  30.1825, 113.5375,  48.6425],
        [ 73.645 ,  10.7975, 100.8275,  30.9325],
        [224.2625, 157.6425, 235.    , 171.735 ],
        [  0.    ,  88.3175,  12.235 , 107.95  ],
        [145.95  , 167.2425, 170.5175, 185.95  ]]),
 array([[ 61.1575,  96.0425,  86.91  , 119.22  ],
        [104.9225,  57.155 , 148.56  , 109.0675]]),
 array([[  5.715 ,  37.755 ,  22.32  ,  54.885 ],
        [ 73.64  , 195.88  , 123.1825, 228.91  ]]),
 array([[ 67.1925,  93.1375, 107.597

In [67]:
for i,fg_img in enumerate(fg_imgs):
    orig_fg += fg_img

In [70]:
for i,bg_img in enumerate(bg_imgs):
    orig_bg += bg_img

In [75]:
np.savez_compressed('training_imgs.npz',fg_patches = orig_fg,bg_patches = orig_bg)
np.savez_compressed('box_labels.npz',*labels)
np.savez_compressed('box_coords.npz',*reordered_bbs)